### Importing the Needed Modules

In [0]:
import sys
import os

# Absolute path to the repo root
PROJECT_ROOT = "/Workspace/Users/thiruvengadamk16@gmail.com/Retail-And-Ecommerce-Analytics-Platform"

# Add repo root to PYTHONPATH (only once)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Debug checks (safe to remove later)
print("Current working directory:", os.getcwd())
print("Repo root added to path:", PROJECT_ROOT in sys.path)
from src.paths import *

In [0]:
import sys
import os

# Absolute path to the repo root
PROJECT_ROOT = "/Workspace/Users/thiruvengadamk16@gmail.com/Retail-And-Ecommerce-Analytics-Platform"

# Add repo root to PYTHONPATH (only once)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Debug checks (safe to remove later)
print("Current working directory:", os.getcwd())
print("Repo root added to path:", PROJECT_ROOT in sys.path)
from src.paths import *

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import sys
sys.path.append('/Workspace/Users/mohammedthoufiq9360@gmail.com/Retail-And-Ecommerce-Analytics-Platform')

from src.paths import SILVER_STORES_PATH, DIM_STORES_PATH
from src.schema_definitions import DIM_STORES_SCHEMA
from src.utils import add_gold_metadata
from delta.tables import DeltaTable

### Querying Silver Stores Table

In [0]:
store_silver_df = spark.read.table(SILVER_STORES_PATH)
store_silver_df.limit(5).display()

### Dim_stores Schema Reference

In [0]:
DIM_STORES_SCHEMA

### Selecting the Needed columns for dim_stores

In [0]:
store_silver_df = store_silver_df.select("store_id", "store_name", "city", "country", "zip_code", "latitude", "longitude", "number_of_employees")

### Creating metadata columns : _created_at and _updated_at

In [0]:
dim_store_df = add_gold_metadata(store_silver_df)

### Creating Dim_stores Table with surrogate key

In [0]:
spark.sql(f"""
        create table if not exists {DIM_STORES_PATH}(
            store_sk long generated always as identity,
            store_id integer,
            store_name string,
            city string,
            country string,
            zip_code string,
            latitude double,
            longitude double,
            number_of_employees integer,
            _created_at timestamp,
            _updated_at timestamp
        ) using delta
    """)

### Updating the Dim_stores Table

In [0]:
dim_store_tbl = DeltaTable.forName(spark, DIM_STORES_PATH)

dim_store_tbl.alias("tgt").merge(
    dim_store_df.alias("src"),
    "tgt.store_id = src.store_id"
).whenMatchedUpdate(set={
    "tgt.store_name": "src.store_name",
    "tgt.city": "src.city",
    "tgt.country": "src.country",
    "tgt.zip_code": "src.zip_code",
    "tgt.latitude": "src.latitude",
    "tgt.longitude": "src.longitude",
    "tgt.number_of_employees": "src.number_of_employees",
    "tgt._updated_at": "src._updated_at"
}).whenNotMatchedInsert(values={
    "tgt.store_id": "src.store_id",
    "tgt.store_name": "src.store_name",
    "tgt.city": "src.city",
    "tgt.country": "src.country",
    "tgt.zip_code": "src.zip_code",
    "tgt.latitude": "src.latitude",
    "tgt.longitude": "src.longitude",
    "tgt.number_of_employees": "src.number_of_employees",
    "tgt._created_at": "src._created_at",
    "tgt._updated_at": "src._updated_at"
}).execute()


In [0]:
spark.read.table(DIM_STORES_PATH).limit(5).display()

In [0]:
spark.read.table(DIM_STORES_PATH).count()